# Install pipelines SDK

#### Please wait till this cell completes and then run next cells. This just need to be run once per active kernel.

In [ ]:
%env RELEASE_VERSION=0.1.18
!pip install https://storage.googleapis.com/ml-pipeline/release/${RELEASE_VERSION}/kfp.tar.gz --upgrade

# Import kfp pkgs

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

# List existing pipeline experiments

In [ ]:
client = kfp.Client()
client.list_experiments()

# Create a Dkube RESNETv2 experiment

In [ ]:
resnet_experiment = client.create_experiment(name='Dkube - Resnetv2 pl')

# Define e2e RESENTv2 Pipeline with Dkube components

In [ ]:
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client

import os
import json
from random import randint

dkube_training_op           = components.load_component_from_file("../components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("../components/serving/component.yaml")
dkube_viewer_op             = components.load_component_from_file('../components/viewer/component.yaml')

@dsl.pipeline(
    name='dkube-cifar-pl',
    description='sample resnetv2 model based catsdogs classification pipeline with dkube components'
)
def d3pipeline(
    #In notebook ACCESS_TOKEN is automatically picked up from env variable
    auth_token  = os.getenv("ACCESS_TOKEN"),
    #By default tf v1.12 image is used here, v1.10, v1.11 or v1.13 can be used. 
    #Or any other custom image name can be supplied.
    #For custom private images, please input username/password
    training_container=json.dumps({'image':'docker.io/ocdr/dkube-datascience-tf-cpu:v1.12', 'username':'', 'password': ''}),
    #Name of the workspace in dkube. Update accordingly if different name is used while creating a workspace in dkube.
    training_program="cifar-workspace",
    #Script to run inside the training container
    training_script="python model.py",
    #Input datasets for training. Update accordingly if different name is used while creating dataset in dkube.    
    training_datasets=json.dumps(["cifar-dataset"]),
    #Request gpus as needed. Val 0 means no gpu, then training_container=docker.io/ocdr/dkube-datascience-tf-cpu:v1.12.
    training_gpus=1,
    #Any envs to be passed to the training program
    training_envs=json.dumps([{"steps": 100}])):

    train       = dkube_training_op(auth_token, training_container,
                                    program=training_program, run_script=training_script,
                                    datasets=training_datasets, ngpus=training_gpus,
                                    envs=training_envs)
    serving     = dkube_serving_op(auth_token, train.outputs['artifact']).after(train)
    inference   = dkube_viewer_op(auth_token, serving.outputs['servingurl'],
                                  'catsdogs', viewtype='inference').after(serving)

# Compile and generate tar ball

In [ ]:
compiler.Compiler().compile(d3pipeline, 'dkube_cifar_pl.tar.gz')
# Upload this generated tarball into the Pipelines UI

# Create and Run pipeline

#### Click the dkube-training stage to see the enhanced Dkube Datascience dashboard, metrics and graphs. Click the dkube-viewer stage for the simple UI to test the model predecitions

In [ ]:
run = client.run_pipeline(resnet_experiment.id, 'resnetv2_classifier_pipeline', 'dkube_cifar_pl.tar.gz', params={})